In [16]:
import pandas as pd
import sqlite3

db_file = 'ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db'

def create_connection(db_file):
        conn = sqlite3.connect(db_file)
        cur = conn.cursor()
        cur.execute("Select name from sqlite_master where type = 'table'")
        return cur.fetchall()


    
create_connection(db_file)   

#we'll see if I even import it like this. By now...

[('personal_info',), ('country_info',), ('career_info',), ('poll_info',)]

In [1]:
%load_ext sql
%config SqlMagic.autocommit = False
%sql sqlite:///ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db

In [2]:
%%sql tables <<

select name
from sqlite_master
where type = 'table'

 * sqlite:///ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db
Done.
Returning data to local variable tables


In [3]:
tables

name
personal_info
country_info
career_info
poll_info


In [4]:
%%sql table1 <<

select country_info.country_code, career_info.normalized_job_code, country_info.rural 
from country_info
join career_info
on career_info.uuid = country_info.uuid



 * sqlite:///ih_datamadpt0420_project_m1/data/raw/raw_data_project_m1.db
Done.
Returning data to local variable table1


In [5]:
data1 = table1.DataFrame()

data1.head()

,country_code,normalized_job_code,rural
0,AT,None,countryside
1,AT,861a9b9151e11362eb3c77ca914172d0,urban
2,AT,None,city
3,AT,049a3f3a2b5f85cb2971ba77ad66e10c,Country
4,AT,f4b2fb1aa40f661488e2782b6d57ad2f,city


In [25]:
#Making the  API

In [8]:
import requests

def response_api(job_code):
    return requests.get(f'http://api.dataatwork.org/v1/jobs/{job_code}').json().get('title')


In [10]:

def jobs_to_dict():
    job_codes_list = list(data1['normalized_job_code'].dropna().unique())
    return {job: response_api(job) for job in job_codes_list}

jobs_dict = jobs_to_dict()

data1['normalized_job_code'] = data1['normalized_job_code'].apply(lambda job: jobs_dict.get(job))


In [11]:
data1.head()

,country_code,normalized_job_code,rural
0,AT,None,countryside
1,AT,Automatic Data Processing Planner,urban
2,AT,None,city
3,AT,Data Coordinator,Country
4,AT,Database Developer,city


In [12]:
data1['rural'] = data1['rural'].str.lower()

data1['rural'] = data1['rural'].str.replace('city','urban').replace('non-rural','urban').replace('countryside','rural').replace('country', 'rural')


data1.head()

,country_code,normalized_job_code,rural
0,AT,None,rural
1,AT,Automatic Data Processing Planner,urban
2,AT,None,urban
3,AT,Data Coordinator,rural
4,AT,Database Developer,urban


In [1]:
import multiprocessing

multiprocessing.cpu_count()

2

In [72]:
import requests
from bs4 import BeautifulSoup
import lxml.html as lh
import re

In [79]:
url = 'https://iban.com/country-codes'
html = requests.get(url).content

soup = BeautifulSoup(html,'lxml')
table = soup.find(id = 'myTable').find_all(['tbody'])

table_list = list(table[0].find_all('td'))


def clean_list(name):
    name1 = re.sub('<td>', '', str(name))
    name2 = re.sub('</td>', '', str(name1))
    return name2

#table_dict = {table_list[i+1]: table_list[i] for i in range(0,len(table_list),4)}
clean_table_list = list(map(clean_list, table_list))

clean_table_list

['Afghanistan',
 'AF',
 'AFG',
 '004',
 'Åland Islands',
 'AX',
 'ALA',
 '248',
 'Albania',
 'AL',
 'ALB',
 '008',
 'Algeria',
 'DZ',
 'DZA',
 '012',
 'American Samoa',
 'AS',
 'ASM',
 '016',
 'Andorra',
 'AD',
 'AND',
 '020',
 'Angola',
 'AO',
 'AGO',
 '024',
 'Anguilla',
 'AI',
 'AIA',
 '660',
 'Antarctica',
 'AQ',
 'ATA',
 '010',
 'Antigua and Barbuda',
 'AG',
 'ATG',
 '028',
 'Argentina',
 'AR',
 'ARG',
 '032',
 'Armenia',
 'AM',
 'ARM',
 '051',
 'Aruba',
 'AW',
 'ABW',
 '533',
 'Australia',
 'AU',
 'AUS',
 '036',
 'Austria',
 'AT',
 'AUT',
 '040',
 'Azerbaijan',
 'AZ',
 'AZE',
 '031',
 'Bahamas (the)',
 'BS',
 'BHS',
 '044',
 'Bahrain',
 'BH',
 'BHR',
 '048',
 'Bangladesh',
 'BD',
 'BGD',
 '050',
 'Barbados',
 'BB',
 'BRB',
 '052',
 'Belarus',
 'BY',
 'BLR',
 '112',
 'Belgium',
 'BE',
 'BEL',
 '056',
 'Belize',
 'BZ',
 'BLZ',
 '084',
 'Benin',
 'BJ',
 'BEN',
 '204',
 'Bermuda',
 'BM',
 'BMU',
 '060',
 'Bhutan',
 'BT',
 'BTN',
 '064',
 'Bolivia (Plurinational State of)',
 'BO',
 'B

In [69]:
test = table_list

lst = []
for i in test:
    i = re.sub('<td>','',str(i))
    i = re.sub('</td>','',str(i))
    lst.append(i)

In [71]:
table_dict2 = {lst[i+1]: lst[i] for i in range(0,len(lst),4)}

table_dict2

{'AF': 'Afghanistan',
 'AX': 'Åland Islands',
 'AL': 'Albania',
 'DZ': 'Algeria',
 'AS': 'American Samoa',
 'AD': 'Andorra',
 'AO': 'Angola',
 'AI': 'Anguilla',
 'AQ': 'Antarctica',
 'AG': 'Antigua and Barbuda',
 'AR': 'Argentina',
 'AM': 'Armenia',
 'AW': 'Aruba',
 'AU': 'Australia',
 'AT': 'Austria',
 'AZ': 'Azerbaijan',
 'BS': 'Bahamas (the)',
 'BH': 'Bahrain',
 'BD': 'Bangladesh',
 'BB': 'Barbados',
 'BY': 'Belarus',
 'BE': 'Belgium',
 'BZ': 'Belize',
 'BJ': 'Benin',
 'BM': 'Bermuda',
 'BT': 'Bhutan',
 'BO': 'Bolivia (Plurinational State of)',
 'BQ': 'Bonaire, Sint Eustatius and Saba',
 'BA': 'Bosnia and Herzegovina',
 'BW': 'Botswana',
 'BV': 'Bouvet Island',
 'BR': 'Brazil',
 'IO': 'British Indian Ocean Territory (the)',
 'BN': 'Brunei Darussalam',
 'BG': 'Bulgaria',
 'BF': 'Burkina Faso',
 'BI': 'Burundi',
 'CV': 'Cabo Verde',
 'KH': 'Cambodia',
 'CM': 'Cameroon',
 'CA': 'Canada',
 'KY': 'Cayman Islands (the)',
 'CF': 'Central African Republic (the)',
 'TD': 'Chad',
 'CL': 'Chil